In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
from scipy.stats import spearmanr

In [2]:
class ImageQualityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        quality = self.data.iloc[idx, 0]  # Quality column
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([quality, authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


In [3]:
class VGG16(nn.Module):
    """VGG16 model for image quality assessment."""

    def __init__(self, num_outputs=2):
        """
        Initializes the VGG16 model.

        Args:
            num_outputs (int): Number of output features. Defaults to 2 (quality and authenticity).
        """
        super(VGG16, self).__init__()
        # Load pre-trained VGG16 model
        self.vgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT)

        # Freeze all layers
        for param in self.vgg16.parameters():
            param.requires_grad = False

        # Modify the classifier
        num_features = self.vgg16.classifier[6].in_features
        self.vgg16.classifier = nn.Sequential(
            *list(self.vgg16.classifier.children())[:-1],  # Remove last layer with 1000 outputs
            nn.Linear(num_features, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_outputs)  # Add new layer with num_out outputs
        )

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.vgg16(x)
    
class QualityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Predict quality and realness
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10, device='cuda'):
    """
    Trains the model.

    Args:
        model (nn.Module): The model to train.
        dataloaders (dict): A dictionary containing the training and validation data loaders.
        criterion (nn.Module): The loss function.
        optimizer (optim.Optimizer): The optimizer.
        num_epochs (int): Number of epochs to train for. Defaults to 10.
        device (str): Device to use for training ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        nn.Module: The trained model.
    """
    model.to(device)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:  # Iterate over training and validation phases
            print(f'{phase} phase')
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            for inputs, labels in dataloaders[phase]:  # Iterate over data in the current phase
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):  # Enable gradients only during training
                    outputs, _ = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f}') # Print loss for the current phase

    print("Finished Training")
    return model

def test_model(model, dataloader, criterion, device='cuda'):

    """
    Tests the model on the test dataset.

    Args:
        model (nn.Module): The trained model.
        dataloader (DataLoader): The test data loader.
        criterion (nn.Module): The loss function.
        device (str): Device to use for testing ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        float: The average loss on the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    model.to(device)
    running_loss = 0.0

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, _ = model(inputs)
            
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

    test_loss = running_loss / len(dataloader.dataset)
    print(f'Test Loss: {test_loss:.4f}')
    return test_loss

def get_predictions(model, dataloader, device)-> tuple[torch.Tensor, torch.Tensor]:
    """
    Get predictions from the model.

    Args:
        model (nn.Module): The trained model.
        dataloader (DataLoader): The data loader.

    Returns:
        tuple: A tuple (predictions, labels) where:
            predictions (torch.Tensor): Predictions from the model.
            labels (torch.Tensor): Ground truth labels.
    """
    model.eval()  # Set the model to evaluation mode
    model.to(device)
    predictions = []
    labels = []

    with torch.no_grad():  # Disable gradient calculation
        for inputs, target in dataloader:
            outputs, _ = model(inputs.to(device))
            predictions.append(outputs)
            labels.append(target)

    #move to cpu and concatenate
    predictions = torch.cat(predictions).cpu()
    labels = torch.cat(labels).cpu()

    return predictions, labels

def evaluate_predictions(predictions: torch.Tensor, labels: torch.Tensor)-> dict:
    """
    Calculate the Spearman correlation coefficient between predictions and labels. 
    Args:
        predictions (torch.Tensor): Predictions from the model.
        labels (torch.Tensor): Ground truth labels.
    Returns:
        dict: Dictionary containing the quality_corr, authenticity_corr, quality_rmse, and authenticity_rm
    """
    # Ensure tensors are on CPU and convert to numpy
    if torch.is_tensor(predictions):
        predictions = predictions.cpu().numpy()
    if torch.is_tensor(labels):
        labels = labels.cpu().numpy()

    quality_corr, _ = spearmanr(predictions[:, 0], labels[:, 0])
    authenticity_corr, _ = spearmanr(predictions[:, 1], labels[:, 1])

    quality_rmse = np.sqrt(np.mean((predictions[:, 0] - labels[:, 0]) ** 2))
    authenticity_rmse = np.sqrt(np.mean((predictions[:, 1] - labels[:, 1]) ** 2))

    return {'quality_corr': quality_corr, 'authenticity_corr': authenticity_corr, 'quality_rmse': quality_rmse, 'authenticity_rmse': authenticity_rmse}



## Training section

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = 'Dataset/AIGCIQA2023/mos_data.csv'

# Create the dataset
dataset = ImageQualityDataset(csv_file=annotations_file, transform=data_transforms)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Test samples: {len(test_dataset)}')

# Create data loaders
batch_size = 64
epochs = 20
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}

# Model initialization for prediction of quality and authenticity (2 outputs)
#model = QualityPredictor()

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss (regression)
# optimizer = optim.Adam(model.regression_head.parameters(), lr=0.001)

# Define the model path
model_path = 'Weights/VGG-16_finetuned_regression.pth'

# quality_predictor_trained= train_model(model, dataloaders, criterion, optimizer, epochs, device)

# # Save the trained model
# torch.save(quality_predictor_trained.state_dict(), model_path)

# Load the trained model
# quality_predictor_trained = QualityPredictor()
# quality_predictor_trained.load_state_dict(torch.load(model_path))


Training samples: 1680
Validation samples: 480
Test samples: 240


## Utilities for pruning 

In [6]:
def compute_feature_map_importance(model, dataloader, device, layer_name) -> tuple[np.ndarray, np.ndarray]:
    """Computes the importance of each feature map in a convolution
    layer by measuring the change in predictions when the feature map is zero
    out.
    
    Returns:
        tuple: (indices, importance_scores) where both are numpy arrays
    """
    #if importance_scores.npy exists, load it
    if os.path.exists('delta_correlation_scores.npy'):
        return np.load('delta_correlation_scores.npy')
    
    model.eval()
    model.to(device)
    delta_corr_scores = []
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]

    # Get baseline predictions
    baseline_predictions = get_predictions(model, dataloader, device)
   
    baseline_results = evaluate_predictions(baseline_predictions[0], baseline_predictions[1])

    baseline_quality_corr = baseline_results['quality_corr']
    
    with torch.no_grad():
        for i in range(layer.out_channels):
            # Create a backup of the weights and bias
            backup_weights = layer.weight[i, ...].clone()
            backup_bias = layer.bias[i].clone() if layer.bias is not None else None

            # Zero out the i-th output channel
            layer.weight[i, ...] = 0
            if layer.bias is not None:
                layer.bias[i] = 0

            # Get predictions with the pruned model
            pruned_predictions = get_predictions(model, dataloader, device)
            pruned_results = evaluate_predictions(pruned_predictions[0], pruned_predictions[1])
            pruned_quality_corr = pruned_results['quality_corr']
            
            # Compute importance score as the change in correlation
            delta_corr = pruned_quality_corr - baseline_quality_corr
            delta_corr_scores.append((i, delta_corr))

            print('Baseline Quality Correlation:', baseline_quality_corr)
            print('Pruned Quality Correlation:', pruned_quality_corr)
            print(f'Feature map {i}: Change in correlation: {delta_corr:.4f}')
            print('------------------')
            # After computing importance, restore weights and bias
            layer.weight[i, ...] = backup_weights
            if layer.bias is not None:
                layer.bias[i] = backup_bias 

    sorted_delta_corr_scores = sorted(delta_corr_scores, key=lambda x: x[1], reverse=True)
    # save np array 
    np.save('delta_quality_correlation_scores.npy', sorted_delta_corr_scores)
    return np.array(sorted_delta_corr_scores)

def remove_noisy_feature_maps(model, dataloader, device, layer_name, sorted_importance_scores, model_path='Weights/pruned_model.pth'):
    """
    Remove noisy feature maps from a convolutional layer based on importance scores.
    Feature maps are zeroed out one by one and kept zeroed only if model performance improves.
    
    Args:
        model: The neural network model
        dataloader: DataLoader for evaluation
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        sorted_importance_scores: List of tuples (channel_index, importance_score) sorted by importance
        
    Returns:
        Dictionary with pruning results and performance metrics
    """
    model.eval()
    model.to(device)
    
    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    
    # Create a backup of the original weights and bias
    original_weights = layer.weight.clone()
    original_bias = layer.bias.clone() if layer.bias is not None else None
    
    # Initialize tracking variables
    removed_features = []
    rmse_history = []
    
    # Get baseline performance
    # Get baseline predictions
    baseline_predictions = get_predictions(model, dataloader, device)
   
    baseline_results = evaluate_predictions(baseline_predictions[0], baseline_predictions[1])

    baseline_quality_corr = baseline_results['quality_corr']
    baseline_authenticity_corr = baseline_results['authenticity_corr']

    baseline_corr = (baseline_quality_corr + baseline_authenticity_corr) / 2
    
    print(f"Baseline correlation: {baseline_corr:.4f}")
    
    # Track the best RMSE  
    # Iterate over the sorted indices and if removing a feature map improves performance, keep it removed
    for idx, (channel_idx, importance_score) in enumerate(sorted_importance_scores):
        channel_idx = int(channel_idx)
        
        # Temporarily zero out this feature map
        layer.weight[channel_idx, ...] = 0
        if layer.bias is not None:
            layer.bias[channel_idx] = 0
        
        # Evaluate model with feature map removed
        predictions = get_predictions(model, dataloader, device)
        results = evaluate_predictions(predictions[0], predictions[1])
        quality_corr = results['quality_corr']
        authenticity_corr = results['authenticity_corr']
        average_new_corr = (quality_corr + authenticity_corr) / 2

        
        print(f"Iteration {idx+1}/{len(sorted_importance_scores)}: " +
              f"Testing removal of channel {channel_idx}, " +
              f"Importance: {importance_score:.4f}, " +
              f"Correlation: {average_new_corr:.4f}")
        
        # Decide whether to keep this feature map removed
        if average_new_corr > baseline_corr:
            baseline_corr = average_new_corr
            removed_features.append(channel_idx)
            print(f"  ✓ IMPROVING: Zeroing out feature map {channel_idx}")
        else:
            # Restore the feature map
            layer.weight[channel_idx, ...] = original_weights[channel_idx, ...]
            if layer.bias is not None:
                layer.bias[channel_idx] = original_bias[channel_idx]
            print(f"  ✗ NOT IMPROVING: Keeping feature map {channel_idx}")
        
    
    # Final statistics
    print("\n------------------")
    print(f"Final correlation: {baseline_corr:.4f}")
    print(f"Improvement over baseline: {baseline_corr - baseline_corr:.4f}")
    print(f"Improvement over baseline: {baseline_corr - baseline_corr:.4f}")
    print(f"Feature reduction: {(len(removed_features)/len(sorted_importance_scores))*100:.1f}%")
    
    # Save the pruned model
    torch.save(model.state_dict(), model_path)
    
    return {
        'removed_features': removed_features,
        'baseline_corr': baseline_corr,
        'final_corr': baseline_corr,
        'improvement': baseline_corr - baseline_corr,
        'reduction_percentage': (len(removed_features)/len(sorted_importance_scores))*100,
        'rmse_history': rmse_history
    }

def remove_negative_impact_corr_feature_maps(model, dataloader, device, layer_name, sorted_corr_importance_scores, model_path='Weights/negative_impact_pruned_model.pth'):
    """
    Remove feature maps that have a negative impact on model performance based on importance scores (impotance score < 0).
    
    Args:
        model: The neural network model
        dataloader: DataLoader for evaluation
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        sorted_importance_scores: List of tuples (channel_index, importance_score) sorted by importance
        
    Returns:
        Dictionary with pruning results and performance metrics
    """
    model.eval()
    model.to(device)
    
    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    
    # Create a backup of the original weights and bias
    original_weights = layer.weight.clone()
    original_bias = layer.bias.clone() if layer.bias is not None else None
    
    # Get baseline predictions
    baseline_predictions = get_predictions(model, dataloader, device)
   
    baseline_results = evaluate_predictions(baseline_predictions[0], baseline_predictions[1])

    baseline_quality_corr = baseline_results['quality_corr']

    # Initialize tracking variables
    removed_features = []
    
    # Iterate over the sorted indices and zero out all the feature maps that have a negative impact (importance > 0)

    for idx, (channel_idx, importance_score) in enumerate(sorted_corr_importance_scores):
        print(f"Iteration {idx} - Channel {channel_idx}: Importance score: {importance_score:.4f}")
        if importance_score > 0:
            channel_idx = int(channel_idx)
            layer.weight[channel_idx, ...] = 0
            if layer.bias is not None:
                layer.bias[channel_idx] = 0
            removed_features.append(channel_idx)

    # Get pruned results
    new_predictions = get_predictions(model, dataloader, device)
   
    new_results = evaluate_predictions(new_predictions[0], new_predictions[1])

    new_quality_corr = new_results['quality_corr']

    # Save the pruned model
    torch.save(model.state_dict(), model_path)

    # Restore original weights for future use
    layer.weight.data.copy_(original_weights)
    if layer.bias is not None:
        layer.bias.data.copy_(original_bias)

    
    return {
        'removed_features': removed_features,
        'baseline_corr': baseline_quality_corr,
        'final_corr': new_quality_corr,
        'improvement': baseline_quality_corr - new_quality_corr,
        'reduction_percentage': (len(removed_features)/len(sorted_corr_importance_scores))*100
    }


In [7]:

# layer to prune
layer = 'features.28'
# Prune the model

model = QualityPredictor()
model.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))
model.eval()
model.to(device)

delta_corr_importance_scores = compute_feature_map_importance(model, val_dataloader, device, layer)

del model

model_2 = QualityPredictor()
model_2.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))

remove_negative_impact_corr_pruning_results = remove_negative_impact_corr_feature_maps(model_2, val_dataloader, device, layer, delta_corr_importance_scores,  model_path='Weights/negative_impact_corr_pruned_model.pth')

del model_2

model_3 = QualityPredictor()
model_3.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))

remove_noisy_feature_maps = remove_noisy_feature_maps(model_3, val_dataloader, device, layer, delta_corr_importance_scores, model_path='Weights/noise_out_corr_pruned_model.pth')

del model_3

/tmp/ipykernel_226553/3114487485.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))
/tmp/ipyker

Iteration 0 - Channel 497.0: Importance score: 0.0082
Iteration 1 - Channel 206.0: Importance score: 0.0073
Iteration 2 - Channel 291.0: Importance score: 0.0068
Iteration 3 - Channel 119.0: Importance score: 0.0055
Iteration 4 - Channel 325.0: Importance score: 0.0045
Iteration 5 - Channel 121.0: Importance score: 0.0043
Iteration 6 - Channel 22.0: Importance score: 0.0040
Iteration 7 - Channel 3.0: Importance score: 0.0036
Iteration 8 - Channel 210.0: Importance score: 0.0036
Iteration 9 - Channel 334.0: Importance score: 0.0035
Iteration 10 - Channel 183.0: Importance score: 0.0033
Iteration 11 - Channel 103.0: Importance score: 0.0032
Iteration 12 - Channel 98.0: Importance score: 0.0031
Iteration 13 - Channel 375.0: Importance score: 0.0031
Iteration 14 - Channel 67.0: Importance score: 0.0030
Iteration 15 - Channel 137.0: Importance score: 0.0029
Iteration 16 - Channel 56.0: Importance score: 0.0029
Iteration 17 - Channel 432.0: Importance score: 0.0029
Iteration 18 - Channel 236

/tmp/ipykernel_226553/3114487485.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_3.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))


Baseline correlation: 0.6066
Iteration 1/512: Testing removal of channel 497, Importance: 0.0082, Correlation: 0.6201
  ✓ IMPROVING: Zeroing out feature map 497
Iteration 2/512: Testing removal of channel 206, Importance: 0.0073, Correlation: 0.6272
  ✓ IMPROVING: Zeroing out feature map 206
Iteration 3/512: Testing removal of channel 291, Importance: 0.0068, Correlation: 0.6338
  ✓ IMPROVING: Zeroing out feature map 291
Iteration 4/512: Testing removal of channel 119, Importance: 0.0055, Correlation: 0.6399
  ✓ IMPROVING: Zeroing out feature map 119
Iteration 5/512: Testing removal of channel 325, Importance: 0.0045, Correlation: 0.6435
  ✓ IMPROVING: Zeroing out feature map 325
Iteration 6/512: Testing removal of channel 121, Importance: 0.0043, Correlation: 0.6446
  ✓ IMPROVING: Zeroing out feature map 121
Iteration 7/512: Testing removal of channel 22, Importance: 0.0040, Correlation: 0.6471
  ✓ IMPROVING: Zeroing out feature map 22
Iteration 8/512: Testing removal of channel 3, Im

In [10]:
baseline_model = QualityPredictor()
baseline_model.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))

baseline_predictions = get_predictions(baseline_model, test_dataloader, device)
baseline_results = evaluate_predictions(baseline_predictions[0], baseline_predictions[1])


remove_negative_impact_corr_pruned_model = QualityPredictor()
remove_negative_impact_corr_pruned_model.load_state_dict(torch.load('Weights/negative_impact_corr_pruned_model.pth'))

remove_negative_impact_corr_predictions = get_predictions(remove_negative_impact_corr_pruned_model, test_dataloader, device)
remove_negative_impact_corr_results = evaluate_predictions(remove_negative_impact_corr_predictions[0], remove_negative_impact_corr_predictions[1])


noise_out_corr_pruned_model = QualityPredictor()
noise_out_corr_pruned_model.load_state_dict(torch.load('Weights/noise_out_corr_pruned_model.pth'))

noise_out_corr_predictions = get_predictions(noise_out_corr_pruned_model, test_dataloader, device)
noise_out_corr_results = evaluate_predictions(noise_out_corr_predictions[0], noise_out_corr_predictions[1])

print('Baseline Model Results:')
print(baseline_results)

print('Remove Negative Impact Correlation Pruned Model Results:')
print(remove_negative_impact_corr_results)

print('Noise Out Correlation Pruned Model Results:')
print(noise_out_corr_results)


/tmp/ipykernel_226553/2702022483.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  baseline_model.load_state_dict(torch.load('Weights/VGG-16_finetuned_regression.pth'))
/t

Baseline Model Results:
{'quality_corr': np.float64(0.6532543967777217), 'authenticity_corr': np.float64(0.6277643709092171), 'quality_rmse': np.float32(11.410385), 'authenticity_rmse': np.float32(10.7469015)}
Remove Negative Impact Correlation Pruned Model Results:
{'quality_corr': np.float64(0.6662902133717598), 'authenticity_corr': np.float64(0.5496953072101946), 'quality_rmse': np.float32(14.828631), 'authenticity_rmse': np.float32(15.409183)}
Noise Out Correlation Pruned Model Results:
{'quality_corr': np.float64(0.6543967777218354), 'authenticity_corr': np.float64(0.5937082241011127), 'quality_rmse': np.float32(18.474422), 'authenticity_rmse': np.float32(18.391764)}


In [11]:
# Create a function that extract the indices of the zeroed out feature maps in a convolutional layer

def get_zeroed_feature_maps(model, layer_name):
    """
    Get the indices of the zeroed out feature maps in a convolutional layer.

    Args:
        model (nn.Module): The neural network model.
        layer_name (str): The name of the convolutional layer.

    Returns:
        list: The indices of the zeroed out feature maps.
    """
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    zeroed_feature_maps = []

    for i, weight in enumerate(layer.weight):
        if torch.all(weight == 0):
            zeroed_feature_maps.append(i)
    zeroed_feature_maps.sort()

    num_zeroed = len(zeroed_feature_maps)

    return zeroed_feature_maps, num_zeroed

# Get the zeroed out feature maps in the 'features.28' layer of the noisy pruned model
_, noisy_num_zeroed = get_zeroed_feature_maps(noise_out_corr_pruned_model, 'features.28')

# Get the zeroed out feature maps in the 'features.28' layer of the negative impact pruned model
_, negative_impact_num_zeroed = get_zeroed_feature_maps(remove_negative_impact_corr_pruned_model, 'features.28')

print(f"Noisy pruned model: {noisy_num_zeroed} zeroed out feature maps")

print(f"Negative impact pruned model: {negative_impact_num_zeroed} zeroed out feature maps")




Noisy pruned model: 176 zeroed out feature maps
Negative impact pruned model: 178 zeroed out feature maps


# Model Evaluation Results

## Test Loss Comparison

| Model | Test Loss | Variation from Baseline | Number of removed features |
|-------|-----------|-------------------------| --------------------------|
| RMSE Baseline | 112.4063 | - | - |
| Best RMSE subset pruned (removed only the individual features with negative impact on RSME if zeroed) | 111.8710 | -0.48% | -185 |
| Best RMSE subset Pruned (iterative zeroing) | 66.3359 | -40.99% | -133 |

## Correlation Results
| Model | Quality Correlation | Variation from Baseline | Number of removed features |
|-------|---------------------|-------------------------| --------------------------|
| Corr Baseline | 0.6532543967777217 | - | - |
| Remove Negative Impact Pruned (removed only the individual features with negative impact on corr if zeroed) | 0.6662902133717598 | +2.00% | -176 |
| Noise Out Pruned (iterative zeroing) | 0.6543967777218354 | +0.17% | -178 |